## 1: Face Recognition, but not evil this time

Using the faces dataset in:

```
from sklearn.datasets import fetch_lfw_people
faces = fetch_lfw_people(min_faces_per_person=60)
```

If you use the `faces.target` and `faces.target_names` attributes, you can build a facial recognition algorithm.

Use sklearn **gridsearch** (or an equivalent, like random search) to optimize the model for accuracy. Try both a SVM-based classifier and a logistic regression based classifier (with a feature pipeline of your choice) to get the best model. You should have at least 80% accuracy.

In [11]:
from sklearn.datasets import fetch_lfw_people
from sklearn.model_selection import train_test_split

faces = fetch_lfw_people(min_faces_per_person=60)

In [12]:
import numpy as np 
import pandas as pd 

# Preparing X_train and Y_train to train the model 
#80 % training data 
X_train, X_test, y_train, y_test = train_test_split(
faces.data, faces.target,test_size=0.2)

#N values will go to the holdout to see the final result
N=50
holdout_X=X_test[-N:]
holdout_y=y_test[-N:]

X_test=X_test[:-N]
y_test=y_test[:-N]





In [13]:
from sklearn.svm import SVC # "Support vector classifier"
from sklearn.model_selection import GridSearchCV
from sklearn.linear_model import LogisticRegression


param_grid = {
            # from 0.001 to 1000
            "C":np.logspace(-3,3,7),
            'fit_intercept':[True,False ]
             }
             
logreg=LogisticRegression(multi_class='multinomial')


grid = GridSearchCV(logreg, param_grid,cv=10)
grid.fit(X_train, y_train)
model = grid.best_estimator_
grid.best_params_

_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)
C:\Users\kevin\Anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:764: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)
C:\Users\kevin\Anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:764: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solv

{'C': 0.001, 'fit_intercept': False}

In [14]:
import matplotlib.pyplot as plt

y_pred = model.fit(X_train, y_train).predict(X_test)
acc=(y_pred==y_test).sum()/len(y_pred)*100


C:\Users\kevin\Anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:764: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


In [15]:
from sklearn.svm import SVC # "Support vector classifier"




param_grid2 = {
            # from 0.001 to 1000
            "C":np.logspace(-3,3,7),
            'kernel':['linear'],
            'gamma':[0.1,1]
             }
svc=SVC()

g=np.arange(1e-4,1e-2,0.0001)

grid2 = GridSearchCV(svc, param_grid2,cv=5,verbose=True)
grid2.fit(X_train, y_train)
model2 = grid2.best_estimator_
grid2.best_params_



[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
Fitting 5 folds for each of 14 candidates, totalling 70 fits
[Parallel(n_jobs=1)]: Done  70 out of  70 | elapsed:  4.2min finished


{'C': 0.001, 'gamma': 0.1, 'kernel': 'linear'}

In [16]:

y_pred = model.fit(X_train, y_train).predict(X_test)
acc=(y_pred==y_test).sum()/len(y_pred)*100


y_pred2 = model2.fit(X_train, y_train).predict(X_test)
acc2=(y_pred2==y_test).sum()/len(y_pred)*100


C:\Users\kevin\Anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:764: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


In [17]:
print(F"{acc.round(5)}% accurate \nModel: {model}\n")

print(F"{acc2.round(5)}% accurate \nModel: {model2}\n")


87.27273% accurate 
Model: LogisticRegression(C=0.001, fit_intercept=False, multi_class='multinomial')

86.36364% accurate 
Model: SVC(C=0.001, gamma=0.1, kernel='linear')



In [ ]:
# acc on test data for both predictors 
#best one is 
#LogisticRegression(C=0.001, fit_intercept=False, multi_class='multinomial')

In [18]:
y_pred_h = model.fit(X_train, y_train).predict(holdout_X)
acc_h=(y_pred_h==holdout_y).sum()/len(y_pred_h)*100


y_pred2_h = model2.fit(X_train, y_train).predict(holdout_X)
acc2_h=(y_pred2_h==holdout_y).sum()/len(y_pred2_h)*100

C:\Users\kevin\Anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:764: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


In [19]:
print(F"{acc_h.round(5)}% accurate \nModel: {model}\n")

print(F"{acc2_h.round(5)}% accurate \nModel: {model2}\n")


80.0% accurate 
Model: LogisticRegression(C=0.001, fit_intercept=False, multi_class='multinomial')

88.0% accurate 
Model: SVC(C=0.001, gamma=0.1, kernel='linear')



In [ ]:
#% acc for both predictors on holdout data
#best one is Model: SVC(C=0.001, gamma=0.1, kernel='linear')

#conclusion they are both good but svc gave is better result for 
# the holdout data

# 2: Bag of Words, Bag of Popcorn

By this point, you are ready for the [Bag of Words, Bag of Popcorn](https://www.kaggle.com/c/word2vec-nlp-tutorial/data) competition. 

Use NLP feature pre-processing (using, SKLearn, Gensim, Spacy or Hugginface) to build the best classifier you can. Use a  feature pipeline, and gridsearch for your final model.

A succesful project should get 90% or more on a **holdout** dataset you kept for yourself.

In [20]:
import pandas as pd 
import numpy as np 
df=pd.read_csv('labeledTrainData.tsv',sep='\t')
#df=df.sample(frac=0.001).reset_index()
original=df.copy()
df

,id,sentiment,review
0,5814_8,1,With all this stuff going down at the moment w...
1,2381_9,1,"\The Classic War of the Worlds\"" by Timothy Hi..."
2,7759_3,0,The film starts with a manager (Nicholas Bell)...
3,3630_4,0,It must be assumed that those who praised this...
4,9495_8,1,Superbly trashy and wondrously unpretentious 8...
...,...,...,...
24995,3453_3,0,It seems like more consideration has gone into...
24996,5064_1,0,I don't believe they made this film. Completel...
24997,10905_3,0,"Guy is a loser. Can't get girls, needs to buil..."
24998,10194_3,0,This 30 minute documentary Buñuel made in the ...


In [21]:
from sklearn.decomposition import PCA
import sklearn.feature_extraction.text as text
from sklearn import cluster
import seaborn as sns 
from sklearn.cluster import KMeans
from nltk.corpus import stopwords
from  nltk.stem import SnowballStemmer
import matplotlib.pyplot as plt 
from sklearn import preprocessing
import spacy 
import en_core_web_sm
from nltk.stem import WordNetLemmatizer 
import gensim
import gensim.downloader as model_api
import re

# Cleaning the data 
TEXT_CLEANING_RE = "@\S+|https?:\S+|http?:\S|[^A-Za-z]+"

# to remove the stop words
stop_words = stopwords.words("english")




stemmer = SnowballStemmer("english")
lemmatizer = WordNetLemmatizer() 

#df.review= df.review.apply(lambda x: remove_emoji(x))


#preprocess fontion 
def preprocess(text, stem=False,lem=True):
    # Remove link,user and special characters
    text = re.sub(TEXT_CLEANING_RE, ' ', str(text).lower()).strip()
    tokens = []
    for token in text.split():
        if token not in stop_words:
            if stem:
                tokens.append(stemmer.stem(token))
            if lem:
                tokens.append(lemmatizer.lemmatize(token))
            else:
                tokens.append(token)
    return " ".join(tokens)

# cleaning the data for every rows
df.review= df.review.apply(lambda x: preprocess(x))

#removing empty rows for original data and clean data
original=original.drop((df.review.loc[(df.review.str.len()==0)]).index)
original=original.reset_index().drop('index',axis=1)

df=df.drop((df.review.loc[(df.review.str.len()==0)]).index)
df=df.reset_index().drop('index',axis=1)


In [22]:
# Looking at a random row

i=np.random.randint(len(df))
print('Origonal comment')
display(original.review.iloc[i])
display(df.review.iloc[i])


Origonal comment


"Elizabeth Rohm was the weakest actress of all the Law and Order ADA's and her acting is even worse here. Her attempts at a Texas accent are amateurish and unrealistic. Nor can she adequately summon the intense emotions needed to play the mother of a kidnapped child; at times while her daughter is missing she manages to sound only vaguely annoyed, as if she can't remember where she left her keys.<br /><br />This is an important true story, so it's too bad that the awful acting of the lead actress distracts so much from the message. The rest of the cast is talented enough, but they just can't overcome Rohm's tendency to simply lay on a particularly thick imitation of a Southern drawl whenever actual acting is required."

'elizabeth rohm weakest actress law order ada acting even worse attempt texas accent amateurish unrealistic adequately summon intense emotion needed play mother kidnapped child time daughter missing manages sound vaguely annoyed remember left key br br important true story bad awful acting lead actress distracts much message rest cast talented enough overcome rohm tendency simply lay particularly thick imitation southern drawl whenever actual acting required'

In [23]:
import gensim
import spacy 
import spacy 
import en_core_web_sm
####################################
# Loading pretrained word embadding
####################################


# Load pretrained word embeddings glove (Gensim)
word_vectors = model_api.load("glove-wiki-gigaword-300")

# Load pretrained word embeddings English (spacy)
word_vectors_spacy = en_core_web_sm.load()



In [24]:

from sklearn.feature_extraction.text import TfidfVectorizer

#glove 
#-----------------------------------------------------------------
def soft_get(w):
    try:
        return word_vectors[w]
    except KeyError:
        return np.zeros(word_vectors.vector_size)
def map_vectors(row):
    if len(row.loc[row.notna()])!=0:
        return np.sum(row.loc[row.notna()].apply(soft_get))
    else :
        return np.zeros(word_vectors.vector_size)

def glove_emb(df):
    words = pd.DataFrame(df.review.str.split().tolist())
    emb = pd.DataFrame(words.apply(map_vectors, axis=1).tolist())
    return emb 


#spacy 
#-----------------------------------------------------------------

def vect(s):
    return word_vectors_spacy(s).vector

def spacy_emb(df):
    a=pd.DataFrame((df.review.apply(vect)))
    emb=pd.DataFrame(a.apply(pd.Series).stack().tolist())
    return emb 
#-----------------------------------------------------------------


# TF-Idf
#---------------------------------------------------
def tf_idf(df):
    #tfidf = text.TfidfVectorizer(max_features=10000,norm='l2') #ok 
    tfidf = text.TfidfVectorizer(max_features=20000) #ok 

    #ignore terms that appear in less than 1% of the documents
    #ignore terms that appear in more than 98% of the documents

    tfidf.fit(df.review)
    t = tfidf.transform(df.review)
    #just making a matrix 

    t=(t.todense())
    return t
    #---------------------------------------------------

def df_emb(df,wd):
    if wd=='spacy':
        return spacy_emb(df)
    elif wd=='glove':
        return glove_emb(df)
    elif wd=='tf':
        return tf_idf(df)







In [25]:
from sklearn.svm import SVC # "Support vector classifier"
from sklearn.model_selection import GridSearchCV
from sklearn.linear_model import LogisticRegression





In [26]:
import seaborn as sns
# Preparing X_train and Y_train to train the model 
#80 % training data 

emb=df_emb(df,wd='tf')

X_train2, X_test2, y_train2, y_test2 = train_test_split(
emb,df.sentiment,test_size=0.20)

#N values will go to the holdout to see the final result
N=1000
holdout_X2=X_test2[-N:]
holdout_y2=y_test2[-N:]

X_test2=X_test2[:-N]
y_test2=y_test2[:-N]

In [27]:
X_train2.shape

(20000, 20000)

In [28]:
from sklearn.preprocessing import Normalizer
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import BaggingClassifier

norm=Normalizer()

In [29]:
from sklearn.preprocessing import StandardScaler, KBinsDiscretizer
from sklearn.pipeline import Pipeline
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import Normalizer
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import BaggingClassifier


pipe = Pipeline([
    ('normalizer',Normalizer()),
    ('Logit', LogisticRegression(max_iter=100))
    
])

param_grid = [
    {
    'normalizer__norm':['l1', 'l2', 'max'], 
    "Logit__C":np.arange(90,150,10),

    
    #'Logit__fit_intercept':[True,False ],
    #'Logit__solver':['newton-cg', 'lbfgs','liblinear','sag','saga']
}]

grid3 = GridSearchCV(pipe, param_grid,cv=2,scoring='accuracy', verbose=10)
grid3.fit(X_train2, y_train2)

model3 = grid3.best_estimator_
display(grid3.best_params_)


model = grid3.best_estimator_

Fitting 2 folds for each of 18 candidates, totalling 36 fits
[CV] Logit__C=90, normalizer__norm=l1 ................................
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
C:\Users\kevin\Anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:764: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)
[CV] .... Logit__C=90, normalizer__norm=l1, score=0.875, total=  46.6s
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:   46.9s remaining:    0.0s
[CV] Logit__C=90, normalizer__norm=l1 ................................
C:\Users\kevin\Anaconda3\lib\site-

{'Logit__C': 140, 'normalizer__norm': 'l1'}

In [30]:
y_pred3 = model3.fit(X_train2, y_train2).predict(X_test2)
acc=(y_pred3==y_test2).sum()/len(y_pred3)*100
print(acc)
#We have an accuracy of 89.65% on the test data 

C:\Users\kevin\Anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:764: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)
89.64999999999999


In [32]:



y_predf = model.fit(X_train2, y_train2).predict(holdout_X2)
acc=(y_predf==holdout_y2).sum()/len(holdout_y2)*100
print(acc)

C:\Users\kevin\Anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:764: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)
89.4


In [ ]:
# 89.4 % on the holdout data

In [33]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.pipeline import Pipeline
from sklearn.ensemble import RandomForestClassifier

pipe = Pipeline([
    ('Forest', RandomForestClassifier())

])



param_grid = [
    {
    #"Forest__criterion":['gini', 'entropy'],
    'Forest__n_estimators':np.arange(200,500,100)
}]

grid3 = GridSearchCV(pipe, param_grid,cv=2,scoring='accuracy', verbose=10)
grid3.fit(X_train2, y_train2)

model3 = grid3.best_estimator_
display(grid3.best_params_)




Fitting 2 folds for each of 3 candidates, totalling 6 fits
[CV] Forest__n_estimators=200 ........................................
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[CV] ............ Forest__n_estimators=200, score=0.841, total= 3.6min
[CV] Forest__n_estimators=200 ........................................
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:  3.6min remaining:    0.0s
[CV] ............ Forest__n_estimators=200, score=0.852, total= 3.7min
[CV] Forest__n_estimators=300 ........................................
[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:  7.3min remaining:    0.0s
[CV] ............ Forest__n_estimators=300, score=0.840, total= 5.5min
[CV] Forest__n_estimators=300 ........................................
[Parallel(n_jobs=1)]: Done   3 out of   3 | elapsed: 12.8min remaining:    0.0s
[CV] ............ Forest__n_estimators=300, score=0.853, total= 5.1min
[CV] Forest__n_estimators=400 ......................

{'Forest__n_estimators': 400}

In [ ]:
# the accuracy on the test data is lower. so the best model would be 
#{'Logit__C': 140, 'normalizer__norm': 'l1'} with 89.4% on the holdout data